<a href="https://colab.research.google.com/github/olegdenko/Coffee-Spoon-Manager/blob/olegdenko/Hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW5

In [68]:
import os
import pandas as pd
import numpy as np
from scipy.stats import entropy, skew, kurtosis
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Підготовка даних:
За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах.

In [69]:
# Шлях до папки з даними у Google Drive
data_folder = "/content/drive/My Drive/homework"

# Створення порожнього списку для зберігання даних
data = []

# Проходимося по кожній папці
for activity_folder in os.listdir(data_folder):
    # Формуємо шлях до кожної вкладеної папки з даними
    activity_path = os.path.join(data_folder, activity_folder)
    # Якщо це папка
    if os.path.isdir(activity_path):
        # Створюємо пустий список для зберігання даних з поточної папки
        activity_data = []
        # Проходимося по кожному файлу в папці
        for file_name in os.listdir(activity_path):
            # Формуємо шлях до кожного файлу
            file_path = os.path.join(activity_path, file_name)
            # Зчитуємо дані з файлу і додаємо до списку activity_data
            df = pd.read_csv(file_path)
            # Додаємо дані з відповідною міткою класу
            df['activity'] = activity_folder
            activity_data.append(df)
        # Об'єднуємо всі дані з поточної папки в один DataFrame
        activity_df = pd.concat(activity_data, ignore_index=True)
        # Додаємо DataFrame з поточної папки до загального списку data
        data.append(activity_df)

# Об'єднуємо всі дані з усіх папок в один DataFrame
full_data = pd.concat(data, ignore_index=True)

# Виведення декількох рядків для перевірки
full_data

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-6.698976,-7.776367,5.104438,running
1,1.513135,13.038822,3.476381,running
2,-22.998701,31.598675,-26.297910,running
3,-5.942408,2.935291,-11.113884,running
4,7.886501,14.863204,-8.508993,running
...,...,...,...,...
193855,-1.005565,-2.700659,-0.545878,walking
193856,3.835511,-13.172897,2.097321,walking
193857,-3.945644,-17.257406,-20.973207,walking
193858,-0.344765,-9.083601,1.297657,walking


# Розрахунок часових ознак (time domain features):
Розрахувати характеристики, які дозволяють описати часовий характер даних, такі як середнє, медіана, дисперсія, тощо.

In [70]:
# Функція для розрахунку часових ознак
def calculate_time_features(data):
    # Ініціалізація списку для зберігання результатів
    time_features = {}

    # Розрахунок часових ознак для кожного виміру (X, Y, Z)
    for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']:
        # Отримання даних для певного виміру
        axis_data = data[axis]

        # Розрахунок часових ознак
        mean = np.mean(axis_data)
        median = np.median(axis_data)
        min_val = np.min(axis_data)
        max_val = np.max(axis_data)

        # Додавання результатів до словника
        time_features[f'{axis}_mean'] = mean
        time_features[f'{axis}_median'] = median
        time_features[f'{axis}_min'] = min_val
        time_features[f'{axis}_max'] = max_val

    return time_features

# Використання функції для розрахунку часових ознак для даних
time_features_example = calculate_time_features(full_data)

# Додаємо часові ознаки до даних
time_features_df = full_data.apply(lambda row: pd.Series(calculate_time_features(row), index=None), axis=1)

# Об'єднуємо часові ознаки з оригінальним DataFrame
full_data_with_time_features = pd.concat([full_data, time_features_df], axis=1)

print("Часові ознаки для першого зразка даних (X, Y, Z):")
time_features_example
print("\nДані з доданими часовими ознаками:")
full_data_with_time_features.head()

Часові ознаки для першого зразка даних (X, Y, Z):

Дані з доданими часовими ознаками:


,accelerometer_X,accelerometer_Y,accelerometer_Z,activity,accelerometer_X_mean,accelerometer_X_median,accelerometer_X_min,accelerometer_X_max,accelerometer_Y_mean,accelerometer_Y_median,accelerometer_Y_min,accelerometer_Y_max,accelerometer_Z_mean,accelerometer_Z_median,accelerometer_Z_min,accelerometer_Z_max
0,-6.698976,-7.776367,5.104438,running,-6.698976,-6.698976,-6.698976,-6.698976,-7.776367,-7.776367,-7.776367,-7.776367,5.104438,5.104438,5.104438,5.104438
1,1.513135,13.038822,3.476381,running,1.513135,1.513135,1.513135,1.513135,13.038822,13.038822,13.038822,13.038822,3.476381,3.476381,3.476381,3.476381
2,-22.998701,31.598675,-26.297910,running,-22.998701,-22.998701,-22.998701,-22.998701,31.598675,31.598675,31.598675,31.598675,-26.297910,-26.297910,-26.297910,-26.297910
3,-5.942408,2.935291,-11.113884,running,-5.942408,-5.942408,-5.942408,-5.942408,2.935291,2.935291,2.935291,2.935291,-11.113884,-11.113884,-11.113884,-11.113884
4,7.886501,14.863204,-8.508993,running,7.886501,7.886501,7.886501,7.886501,14.863204,14.863204,14.863204,14.863204,-8.508993,-8.508993,-8.508993,-8.508993


# Розбиття на навчальний і тестовий набори:
Розділити дані на навчальний і тестовий набори для оцінки точності моделей.

In [71]:
# Розділення на ознаки (X) і цільову змінну (y)
X = full_data_with_time_features.drop(columns=['activity'])

y = full_data_with_time_features['activity']

# Розділення на навчальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Вивід розмірів навчального і тестового наборів
print("Розмір навчального набору:", X_train.shape[0])
print("Розмір тестового набору:", X_test.shape[0])

Розмір навчального набору: 155088
Розмір тестового набору: 38772


# Навчання моделей:
Використати алгоритми класифікації, такі як SVM та випадковий ліс, для навчання на навчальних даних.

# Навчання моделей SVM:

In [72]:
# Ініціалізація моделі SVM
svm_model = SVC()
X_train.dropna(inplace=True)

# Навчання моделі на навчальних даних
svm_model.fit(X_train, y_train)

SVC()

# Навчання моделі випадкового лісу:

In [73]:
# Ініціалізація моделі випадкового лісу
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# Оцінка моделей:
Оцінити точність моделей за допомогою метрик, таких як classification report, який надає докладну інформацію про класифікаційні показники моделей.

In [74]:
# Передбачення на тестових даних за допомогою моделі SVM
svm_predictions = svm_model.predict(X_test)

# Оцінка точності моделі SVM
print("Classification Report for SVM Model:")
print(classification_report(y_test, svm_predictions))

# Передбачення на тестових даних за допомогою моделі випадкового лісу
rf_predictions = rf_model.predict(X_test)

# Оцінка точності моделі випадкового лісу
print("Classification Report for Random Forest Model:")
print(classification_report(y_test, rf_predictions))

# Перехресна перевірка моделі випадкового лісу
cv_scores = cross_val_score(rf_model, X, y, cv=5)
print("Cross-Validation Scores for Random Forest Model:", cv_scores)
print("Average Cross-Validation Score:", np.mean(cv_scores))

Classification Report for SVM Model:
              precision    recall  f1-score   support

        idle       0.96      0.99      0.97      6178
     running       0.93      0.90      0.92     20510
      stairs       1.00      0.00      0.01      1022
     walking       0.80      0.90      0.85     11062

    accuracy                           0.89     38772
   macro avg       0.92      0.70      0.68     38772
weighted avg       0.90      0.89      0.88     38772

Classification Report for Random Forest Model:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6178
     running       1.00      1.00      1.00     20510
      stairs       1.00      1.00      1.00      1022
     walking       1.00      1.00      1.00     11062

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772

Cross-Validation Scores for Random Forest Mode